In [1]:
import xarray as xr
import numpy as np
from scipy import stats
import get_Amaps as amap

## Get statistics from default ensemble for the modern period

In [2]:
# get default 5-member ensemble for tsmin, tsmax, prec
prec_modern = xr.open_mfdataset("../NINT_ensemble/ij_concat_files/1980-2014_prec_E213f10*F40oQ40_3.nc", combine="nested", concat_dim="ensemble_member")
tsmin_modern = xr.open_mfdataset("../NINT_ensemble/ij_concat_files/1980-2014_tsmin_E213f10*F40oQ40_3.nc", combine="nested", concat_dim="ensemble_member")
tsmax_modern = xr.open_mfdataset("../NINT_ensemble/ij_concat_files/1980-2014_tsmax_E213f10*F40oQ40_3.nc", combine="nested", concat_dim="ensemble_member")

In [3]:
# calculate r1mm, r10mm, t40C, and fd
r1mm_modern = amap.r1mm(prec_modern, "prec", 1).mean("time") # saying 1 ensemble member in argument because I do not want the ensemble average. It is actually 5 members
r10mm_modern = amap.r10mm(prec_modern, "prec", 1).mean("time")
t40C_modern = amap.t40C(tsmax_modern, "tsmax", 1).mean("time")
fd_modern = amap.fd(tsmin_modern, "tsmin", 1).mean("time")

<string>:6: FutureWarning: 'Y' is deprecated and will be removed in a future version. Please use 'YE' instead of 'Y'.
<string>:6: FutureWarning: 'Y' is deprecated and will be removed in a future version. Please use 'YE' instead of 'Y'.
<string>:6: FutureWarning: 'Y' is deprecated and will be removed in a future version. Please use 'YE' instead of 'Y'.
<string>:6: FutureWarning: 'Y' is deprecated and will be removed in a future version. Please use 'YE' instead of 'Y'.


In [4]:
# get standard deviations
r1mm_modern_std = r1mm_modern.std(dim="ensemble_member")
r10mm_modern_std = r10mm_modern.std(dim="ensemble_member")
t40C_modern_std = t40C_modern.std(dim="ensemble_member")
fd_modern_std = fd_modern.std(dim="ensemble_member")

In [20]:
# retrieve default ensemble means for A maps
r1mm_modern_mean = xr.open_dataset("processed_ens/r1mm_ensavg_modern_1A.nc")
r10mm_modern_mean = xr.open_dataset("processed_ens/r10mm_ensavg_modern_1A.nc")
t40C_modern_mean = xr.open_dataset("processed_ens/t40C_ensavg_modern_1A.nc")
fd_modern_mean = xr.open_dataset("processed_ens/fd_ensavg_modern_1A.nc")

## Get statistics from default ensemble for the modern-preindustrial change

In [21]:
# get default 5-member ensemble for tsmin, tsmax, prec
prec_pi = xr.open_mfdataset("../NINT_ensemble/ij_concat_files/1850-1900_prec_E213f10*F40oQ40_3.nc", combine="nested", concat_dim="ensemble_member").sel(time=slice("1851", "1900"))
tsmin_pi = xr.open_mfdataset("../NINT_ensemble/ij_concat_files/1850-1900_tsmin_E213f10*F40oQ40_3.nc", combine="nested", concat_dim="ensemble_member").sel(time=slice("1851", "1900"))
tsmax_pi = xr.open_mfdataset("../NINT_ensemble/ij_concat_files/1850-1900_tsmax_E213f10*F40oQ40_3.nc", combine="nested", concat_dim="ensemble_member").sel(time=slice("1851", "1900"))

In [22]:
# calculate r1mm, r10mm, t40C, and fd
r1mm_pi = amap.r1mm(prec_pi, "prec", 1).mean("time") # saying 1 ensemble member in argument because I do not want the ensemble average. It is actually 5 members
r10mm_pi = amap.r10mm(prec_pi, "prec", 1).mean("time")
t40C_pi = amap.t40C(tsmax_pi, "tsmax", 1).mean("time")
fd_pi = amap.fd(tsmin_pi, "tsmin", 1).mean("time")

<string>:6: FutureWarning: 'Y' is deprecated and will be removed in a future version. Please use 'YE' instead of 'Y'.
<string>:6: FutureWarning: 'Y' is deprecated and will be removed in a future version. Please use 'YE' instead of 'Y'.
/home/mypcpls/anaconda3/envs/geo/lib/python3.12/site-packages/xarray/core/indexing.py:1621: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  return self.array[key]
<string>:6: FutureWarning: 'Y' is deprecated and will be removed in a future version. Please use 'YE' instead of 'Y'.
/home/mypcpls/anaconda3/envs/geo/lib/python3.12/site-packages/xarray/core/indexing.py:1621: PerformanceWarning: Slicing is producing a large chunk.

In [23]:
# get modern preindustrial changes
r1mm_mpc = r1mm_modern - r1mm_pi
r10mm_mpc = r10mm_modern - r10mm_pi
t40C_mpc = t40C_modern - t40C_pi
fd_mpc = fd_modern - fd_pi

In [24]:
# get standard deviations
r1mm_mpc_std = r1mm_mpc.std(dim="ensemble_member")
r10mm_mpc_std = r10mm_mpc.std(dim="ensemble_member")
t40C_mpc_std = t40C_mpc.std(dim="ensemble_member")
fd_mpc_std = fd_mpc.std(dim="ensemble_member")

In [25]:
# retrieve default ensemble mean mpchange maps
r1mm_mpc_mean = xr.open_dataset("processed_ens/r1mm_1D.nc")
r10mm_mpc_mean = xr.open_dataset("processed_ens/r10mm_1D.nc")
t40C_mpc_mean = xr.open_dataset("processed_ens/t40C_1D.nc")
fd_mpc_mean = xr.open_dataset("processed_ens/fd_1D.nc")

# Significance tests

In [26]:
def calculate_p_values(t_stat, df=4): # df=4 because 5 + n - 2, where 5 is default ensemble members and n is model ensemble members (1) 
    return 2 * (1 - stats.t.cdf(np.abs(t_stat), df=df))

def perform_t_test(mean_default, std_default, ds2, varname, n=1, alpha=0.05):
    """
    two-tailed t-test for each grid cell comparing model (ds2) to the ensemble mean of default.

    Parameters:
    - mean_default: default (E2.1-G NINT) ensemble average
    - std_default: default ensemble standard deviation
    - ds2: model to compare with default
    - n: Number of members in ds2 (1)
    - alpha: Significance level for the test (default is 0.05).

    Returns:
    - p_values_da: DataArray of p-values for each grid cell.
    - significant_cells_da: DataArray indicating significant cells (True/False).
    """
    
    # calculate the t-statistic
    n=1
    alpha=0.05
    t_stat = (mean_default - ds2) / (std_default / np.sqrt(5)) # 5 is number of ensemble members in default
    # calculate the p-value for each grid cell
    p_values = xr.apply_ufunc(
        calculate_p_values,
        t_stat,
        vectorize=True,
        dask='allowed',  # use dask if t_stat is a dask array
    )
    
    # create a boolean T/F for significant cells (p < alpha)
    significant_cells = p_values < alpha
    # convert p_values and significant_cells to data arrays with coordinates
    p_values = p_values.rename({varname: "pval"})

    return p_values, significant_cells # return two datasets, one with p value and another with boolean mask for significance

## Modern period differences - significance test 

In [27]:
# significance maps A
for cid in ["R10mm", "R1mm", "T40C", "fd"]:
    cid_lower = "t40C" if cid == "T40C" else cid.lower()
    if (cid == "R10mm") | (cid == "R1mm"):
        folder = "WetAndDry"
    else:
        folder = "HeatAndCold"
    for mid in [2, 3, 4, 5, 6, 8, 10]:
        dsA = xr.open_dataset(f"data4maps/{folder}/{cid}/{cid_lower}_{mid}A.nc")
        default_ens_mean = f"{cid_lower}_modern_mean"
        default_ens_std = f"{cid_lower}_modern_std"
        p_values, significant_cells = perform_t_test(locals()[default_ens_mean], locals()[default_ens_std], dsA, cid_lower, 1, 0.05)
        signif_ds = xr.merge([p_values, significant_cells])
        dsA_signif = dsA.where(signif_ds)
        signif_ds.to_netcdf(f"statsignif_maps/{cid_lower}_pvals_{mid}A.nc")
        dsA_signif.to_netcdf(f"statsignif_maps/{cid_lower}_significant_{mid}A.nc")

/home/mypcpls/anaconda3/envs/geo/lib/python3.12/site-packages/dask/core.py:127: RuntimeWarning: divide by zero encountered in divide
  return func(*(_execute_task(a, cache) for a in args))
/home/mypcpls/anaconda3/envs/geo/lib/python3.12/site-packages/dask/core.py:127: RuntimeWarning: invalid value encountered in divide
  return func(*(_execute_task(a, cache) for a in args))
/home/mypcpls/anaconda3/envs/geo/lib/python3.12/site-packages/dask/core.py:127: RuntimeWarning: divide by zero encountered in divide
  return func(*(_execute_task(a, cache) for a in args))
/home/mypcpls/anaconda3/envs/geo/lib/python3.12/site-packages/dask/core.py:127: RuntimeWarning: invalid value encountered in divide
  return func(*(_execute_task(a, cache) for a in args))
/home/mypcpls/anaconda3/envs/geo/lib/python3.12/site-packages/dask/core.py:127: RuntimeWarning: divide by zero encountered in divide
  return func(*(_execute_task(a, cache) for a in args))
/home/mypcpls/anaconda3/envs/geo/lib/python3.12/site-pack

## Modern-preindustrial control differences - significance test 

In [28]:
# significance maps E
for cid in ["R10mm", "R1mm", "T40C", "fd"]:
    cid_lower = "t40C" if cid == "T40C" else cid.lower()
    if (cid == "R10mm") | (cid == "R1mm"):
        folder = "WetAndDry"
    else:
        folder = "HeatAndCold"
    for mid in [2, 3, 4, 5, 6, 8, 10]:
        dsD = xr.open_dataset(f"data4maps/{folder}/{cid}/{cid_lower}_{mid}D.nc")
        default_ens_mean = f"{cid_lower}_mpc_mean"
        default_ens_std = f"{cid_lower}_mpc_std"
        p_values, significant_cells = perform_t_test(locals()[default_ens_mean], locals()[default_ens_std], dsD, cid_lower, 1, 0.05)
        signif_ds = xr.merge([p_values, significant_cells])
        dsD_signif = dsD.where(signif_ds)
        signif_ds.to_netcdf(f"statsignif_maps/{cid_lower}_pvals_{mid}D.nc")
        dsD_signif.to_netcdf(f"statsignif_maps/{cid_lower}_significant_{mid}D.nc")

/home/mypcpls/anaconda3/envs/geo/lib/python3.12/site-packages/dask/core.py:127: RuntimeWarning: divide by zero encountered in divide
  return func(*(_execute_task(a, cache) for a in args))
/home/mypcpls/anaconda3/envs/geo/lib/python3.12/site-packages/dask/core.py:127: RuntimeWarning: invalid value encountered in divide
  return func(*(_execute_task(a, cache) for a in args))
/home/mypcpls/anaconda3/envs/geo/lib/python3.12/site-packages/dask/core.py:127: RuntimeWarning: divide by zero encountered in divide
  return func(*(_execute_task(a, cache) for a in args))
/home/mypcpls/anaconda3/envs/geo/lib/python3.12/site-packages/dask/core.py:127: RuntimeWarning: invalid value encountered in divide
  return func(*(_execute_task(a, cache) for a in args))
/home/mypcpls/anaconda3/envs/geo/lib/python3.12/site-packages/dask/core.py:127: RuntimeWarning: divide by zero encountered in divide
  return func(*(_execute_task(a, cache) for a in args))
/home/mypcpls/anaconda3/envs/geo/lib/python3.12/site-pack